<a href="https://colab.research.google.com/github/pati-dev/text-multi-label/blob/master/attributes_from_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Boot

In [83]:
!pip install scikit-multilearn

%reload_ext tensorboard
%tensorflow_version 2.x
import datetime
import pandas as pd
import numpy as np
import math

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

import gensim

import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
finalDF = pd.read_csv('/content/drive/My Drive/Search Project/finalDF.csv')
# finalDF.head()

# Remove stop words

In [0]:
stop = stopwords.words('english')

finalDF['clean_text'] = finalDF['text'].str.lower().str.split()
finalDF['clean_text'] = finalDF['clean_text'].apply(lambda x: [item for item in x if item not in stop])

# Word Clouds

In [72]:
plt.figure(figsize=(40,25))

# RestaurantsReservations
subset = finalDF[finalDF.RestaurantsReservations==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("RestaurantsReservations",fontsize=40)
plt.imshow(cloud)

TypeError: ignored

<Figure size 2880x1800 with 0 Axes>

In [0]:
plt.figure(figsize=(40,25))

# RestaurantsGoodForGroups
subset = finalDF[finalDF.RestaurantsGoodForGroups==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("RestaurantsGoodForGroups",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# OutdoorSeating
subset = finalDF[finalDF.OutdoorSeating==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("OutdoorSeating",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# RestaurantsTakeOut
subset = finalDF[finalDF.RestaurantsTakeOut==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("RestaurantsTakeOut",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# GoodForKids
subset = finalDF[finalDF.GoodForKids==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("GoodForKids",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# RestaurantsDelivery
subset = finalDF[finalDF.RestaurantsDelivery==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("RestaurantsDelivery",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# BikeParking
subset = finalDF[finalDF.BikeParking==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("BikeParking",fontsize=40)
plt.imshow(cloud)

In [0]:
plt.figure(figsize=(40,25))

# Wifi
subset = finalDF[finalDF.Wifi==True]
text = subset.text.values
cloud = WordCloud(
                          stopwords=STOPWORDS,
                          background_color='black',
                          collocations=False,
                          width=2500,
                          height=1800
                         ).generate(" ".join(text))
plt.axis('off')
plt.title("Wifi",fontsize=40)
plt.imshow(cloud)

# Data Preparation

In [77]:
train, test = train_test_split(finalDF, test_size=0.30, shuffle=True)
print(train.shape, test.shape)

(1316, 10) (564, 10)


## Method 1: TF-IDF using *sci-kit learn*

In [0]:
x_train = train.text.values
x_test = test.text.values

In [82]:
categories = ['RestaurantsReservations',
       'RestaurantsGoodForGroups', 'OutdoorSeating', 'RestaurantsTakeOut',
       'GoodForKids', 'RestaurantsDelivery', 'BikeParking', 'Wifi']

# Define a pipeline combining a text feature extractor with multi lable classifier
NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    NB_pipeline.fit(x_train, train[category])
    # compute the testing accuracy
    prediction = NB_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing RestaurantsReservations
Test accuracy is 0.6312056737588653
... Processing RestaurantsGoodForGroups
Test accuracy is 0.8333333333333334
... Processing OutdoorSeating
Test accuracy is 0.599290780141844
... Processing RestaurantsTakeOut
Test accuracy is 0.8953900709219859
... Processing GoodForKids
Test accuracy is 0.7411347517730497
... Processing RestaurantsDelivery
Test accuracy is 0.7074468085106383
... Processing BikeParking
Test accuracy is 0.9095744680851063
... Processing Wifi
Test accuracy is 0.6453900709219859


In [85]:
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    SVC_pipeline.fit(x_train, train[category])
    # compute the testing accuracy
    prediction = SVC_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing RestaurantsReservations
Test accuracy is 0.8812056737588653
... Processing RestaurantsGoodForGroups
Test accuracy is 0.8563829787234043
... Processing OutdoorSeating
Test accuracy is 0.7872340425531915
... Processing RestaurantsTakeOut
Test accuracy is 0.9184397163120568
... Processing GoodForKids
Test accuracy is 0.8953900709219859
... Processing RestaurantsDelivery
Test accuracy is 0.7783687943262412
... Processing BikeParking
Test accuracy is 0.9095744680851063
... Processing Wifi
Test accuracy is 0.6985815602836879


In [86]:
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])
for category in categories:
    print('... Processing {}'.format(category))
    # train the model using X_dtm & y
    LogReg_pipeline.fit(x_train, train[category])
    # compute the testing accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))

... Processing RestaurantsReservations
Test accuracy is 0.8563829787234043
... Processing RestaurantsGoodForGroups
Test accuracy is 0.8421985815602837
... Processing OutdoorSeating
Test accuracy is 0.774822695035461
... Processing RestaurantsTakeOut
Test accuracy is 0.9060283687943262
... Processing GoodForKids
Test accuracy is 0.8953900709219859
... Processing RestaurantsDelivery
Test accuracy is 0.7588652482269503
... Processing BikeParking
Test accuracy is 0.9095744680851063
... Processing Wifi
Test accuracy is 0.7074468085106383


# References

[Journey to the center of multi-label classification](https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff)

[Multi Label Text Classification with Scikit-Learn](https://towardsdatascience.com/multi-label-text-classification-with-scikit-learn-30714b7819c5)